In [1]:
import geopandas as gpd
import pandas as pd
import shapely
from shapely.ops import nearest_points
#import numpy as np
#import pyproj

#import matplotlib
#from matplotlib.pyplot import figure
#import matplotlib.pyplot as plt
#from matplotlib import cm
#from matplotlib.ticker import LinearLocator, FormatStrFormatter
#from matplotlib.colors import ListedColormap
#from matplotlib.ticker import MaxNLocator

In [2]:
pd.set_option('display.max_rows', 1000)

In [74]:
def distance_to_nearest(row, geom_union, df1, df2, geom1_col='geometry', geom2_col='geometry', src_column=None):
    """Find the nearest point and return the corresponding value from specified column."""
    # Find the geometry that is closest
    nearest = df2[geom2_col] == nearest_points(row[geom1_col], geom_union)[1]
    # Get the corresponding value from df2 (matching is based on the geometry)
    nearest_id = df2[nearest][src_column].get_values()[0]
    start_point: shapely.geometry.Point = row[geom1_col]#row['geometry']
    end_point_series: pd.Series = df2[nearest][geom2_col]#df2[nearest]['geometry']
    end_point: shapely.geometry.Point = end_point_series.iloc[0]
    dist_to_point = start_point.distance(end_point)
    return nearest_id, dist_to_point

# With this test data, everything works fine

In [4]:
test_start = gpd.GeoDataFrame({"id": pd.Series(["spoint1", "spoint2", "spoint3"],
                                               index=["spoint1", "spoint2", "spoint3"]),
                              "geometry": pd.Series([shapely.geometry.Point(1, 0),
                                                     shapely.geometry.Point(5, 0),
                                                     shapely.geometry.Point(3, 5)
                                                    ],
                                                    index=["spoint1", "spoint2", "spoint3"]
                                                   )}
                             )

In [5]:
test_end = gpd.GeoDataFrame({"id": pd.Series(["epoint1", "epoint2", "epoint3", "epoint4", "epoint5"],
                                             index=["epoint1", "epoint2", "epoint3", "epoint4", "epoint5"]
                                            ),
                             "geometry": pd.Series([shapely.geometry.Point(-3, 3),
                                                    shapely.geometry.Point(-1, 4),
                                                    shapely.geometry.Point(5, 4),
                                                    shapely.geometry.Point(2, 4),
                                                    shapely.geometry.Point(8, 0)
                                                   ],
                                                   index=["epoint1", "epoint2", "epoint3", "epoint4", "epoint5"]
                                                  )
                            })

In [6]:
unary_union = test_end.unary_union

In [7]:
test_start['nearest_id'] = test_start.apply(distance_to_nearest,
                                            geom_union=unary_union,
                                            df1=test_start,
                                            df2=test_end,
                                            src_column='id',
                                            axis=1)
test_start

epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool
epoint1    False
epoint2    False
epoint3    False
epoint4    False
epoint5     True
Name: geometry, dtype: bool
epoint1    False
epoint2    False
epoint3    False
epoint4     True
epoint5    False
Name: geometry, dtype: bool


,id,geometry,nearest_id
spoint1,spoint1,POINT (1 0),"(epoint4, 4.123105625617661)"
spoint2,spoint2,POINT (5 0),"(epoint5, 3.0)"
spoint3,spoint3,POINT (3 5),"(epoint4, 1.4142135623730951)"


# Using the real data, it doesn't work

In [8]:
crimes = pd.read_pickle('crimes-transformed.pkl')
schools = pd.read_pickle('schools-transformed.pkl')

In [9]:
# Only work with a small subset for readability
crimes_extract = crimes.iloc[:5]

In [10]:
crimes_extract

,ID,case number,date,block,iucr,primary type,desc,locdesc,arrest,domestic,...,y coord,year,updated on,lat,lon,location,community,UTMx,UTMy,UTMPoint
0,8265244,HT498458,09/15/2011 08:00:00 AM,040XX N MENARD AVE,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,True,False,...,1926315.0,2011,02/12/2018 03:46:59 PM,41.953965,-87.771918,"(41.95396528, -87.771918163)",PORTAGE PARK,436025.308686,4.644953e+06,POINT (436025.3086861699 4644953.17679981)
1,7357772,HS159408,07/01/2001 10:00:00 AM,056XX W CORNELIA AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,1922746.0,2001,02/18/2010 01:12:55 AM,41.944150,-87.767546,"(41.9441497, -87.767545572)",PORTAGE PARK,436377.940882,4.643860e+06,POINT (436377.940881997 4643860.108357577)
2,7609828,HS413731,07/13/2010 12:00:00 PM,041XX N LONG AVE,0820,THEFT,$500 AND UNDER,PARK PROPERTY,False,False,...,1926816.0,2010,02/04/2016 06:33:39 AM,41.955292,-87.762142,"(41.95529191, -87.762141971)",PORTAGE PARK,436836.850742,4.645093e+06,POINT (436836.8507415869 4645093.21940705)
3,7610439,HS413034,07/16/2010 03:00:00 AM,056XX W SCHOOL ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,1921375.0,2010,02/04/2016 06:33:39 AM,41.940387,-87.767568,"(41.940387482, -87.767567805)",PORTAGE PARK,436372.357517,4.643442e+06,POINT (436372.3575166143 4643442.41025281)
4,7610466,HS414753,07/17/2010 12:25:00 AM,032XX N CICERO AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,ALLEY,True,False,...,1920986.0,2010,02/04/2016 06:33:39 AM,41.939216,-87.746878,"(41.939216164, -87.746877814)",PORTAGE PARK,438086.334871,4.643297e+06,POINT (438086.3348712226 4643297.20981963)


In [11]:
schools.head()

,geometry,SCHOOL_NM,SCHOOL_ID,SCH_ADDR,GRADE_CAT,SCH_TYPE,lon,lat,UNIT_ID,UTMx,UTMy,UTMPoint
UNIT_ID,,,,,,,,,,,,
4610,POINT (-87.5748539059 41.7483660139),MANN,610052,8050 S CHAPPEL AV,ES,Elementary School,-87.574854,41.748366,4610,452204.771983,4.621998e+06,POINT (452204.7719826915 4621997.860158128)
5180,POINT (-87.57284397069999 41.768650177),OKEEFFE,610103,6940 S MERRILL AV,ES,Elementary School,-87.572844,41.768650,5180,452386.881166,4.624249e+06,POINT (452386.881166386 4624248.790945017)
5300,POINT (-87.5829920307 41.768594283),PARKSIDE,610116,6938 S EAST END AV,ES,Elementary School,-87.582992,41.768594,5300,451543.358669,4.624248e+06,POINT (451543.3586693273 4624248.25277694)
5440,POINT (-87.60800008530001 41.740365734),PIRIE,610130,650 E 85TH ST,ES,Elementary School,-87.608000,41.740366,5440,449442.603124,4.621129e+06,POINT (449442.6031235668 4621128.574748854)
5610,POINT (-87.60015764629999 41.764223475),REVERE,610146,1010 E 72ND ST,ES,Elementary School,-87.600158,41.764223,5610,450113.209645,4.623773e+06,POINT (450113.2096448886 4623772.796898597)


In [12]:
schools_unary_union = schools.unary_union

In [13]:
crimes_extract['nearest_id'] = crimes_extract.apply(distance_to_nearest,
                                                    geom_union=schools_unary_union,
                                                    df1=crimes_extract,
                                                    df2=schools,
                                                    geom1_col='UTMPoint',
                                                    geom2_col='UTMPoint',
                                                    src_column='UNIT_ID',
                                                    axis=1
                                                   )
crimes_extract

UNIT_ID
4610    False
5180    False
5300    False
5440    False
5610    False
5670    False
5880    False
6100    False
6350    False
6900    False
1135    False
2123    False
2123    False
1105    False
6320    False
6450    False
6570    False
7190    False
7610    False
5640    False
6490    False
4230    False
4310    False
4430    False
4430    False
4710    False
5430    False
5970    False
7720    False
4330    False
6710    False
2940    False
4650    False
5370    False
5480    False
6750    False
6760    False
2510    False
2590    False
2910    False
2970    False
3260    False
3370    False
3380    False
3520    False
3630    False
3690    False
3760    False
3900    False
3980    False
4250    False
4320    False
4720    False
5390    False
6310    False
6440    False
6930    False
7490    False
7880    False
8021    False
2580    False
2830    False
3040    False
3110    False
3530    False
3720    False
4350    False
4570    False
6880    False
6880    False
7050    Fals

IndexError: ('index 0 is out of bounds for axis 0 with size 0', 'occurred at index 0')

# At least I get the same result when calling the distance_to_nearest function directly

In [14]:
distance_to_nearest(crimes.iloc[0],
                    geom_union=schools_unary_union,
                    df1=crimes,
                    df2=schools,
                    geom1_col='UTMPoint',
                    geom2_col='UTMPoint',
                    src_column='UNIT_ID'
                   )

UNIT_ID
4610    False
5180    False
5300    False
5440    False
5610    False
5670    False
5880    False
6100    False
6350    False
6900    False
1135    False
2123    False
2123    False
1105    False
6320    False
6450    False
6570    False
7190    False
7610    False
5640    False
6490    False
4230    False
4310    False
4430    False
4430    False
4710    False
5430    False
5970    False
7720    False
4330    False
6710    False
2940    False
4650    False
5370    False
5480    False
6750    False
6760    False
2510    False
2590    False
2910    False
2970    False
3260    False
3370    False
3380    False
3520    False
3630    False
3690    False
3760    False
3900    False
3980    False
4250    False
4320    False
4720    False
5390    False
6310    False
6440    False
6930    False
7490    False
7880    False
8021    False
2580    False
2830    False
3040    False
3110    False
3530    False
3720    False
4350    False
4570    False
6880    False
6880    False
7050    Fals

IndexError: index 0 is out of bounds for axis 0 with size 0

# The nearest_points call works for both datasets when I manually pick a slice

In [28]:
print(nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[0])
print(nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[1])
print(nearest_points(test_start.iloc[0]['geometry'], unary_union)[0])
print(nearest_points(test_start.iloc[0]['geometry'], unary_union)[1])

POINT (436025.3086861699 4644953.17679981)
POINT (-87.67202378280001 42.0210669371)
POINT (1 0)
POINT (2 4)


I'm going to store the nearest school point to be used later on.

In [40]:
nearest_school = nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[1]
print(nearest_school)

POINT (-87.67202378280001 42.0210669371)


In [20]:
real_nearest = schools['UTMPoint'] == nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[1]
test_nearest = test_end['geometry'] == nearest_points(test_start.iloc[0]['geometry'], unary_union)[1]

# I think this is where things go south
If I'm reading the output below, the above equality test of the point found by the nearest_points() call doesn't succeed on the real data (i.e. there is no row in real_nearest that has a True value).

In [21]:
print(real_nearest)
print(test_nearest)

UNIT_ID
4610    False
5180    False
5300    False
5440    False
5610    False
5670    False
5880    False
6100    False
6350    False
6900    False
1135    False
2123    False
2123    False
1105    False
6320    False
6450    False
6570    False
7190    False
7610    False
5640    False
6490    False
4230    False
4310    False
4430    False
4430    False
4710    False
5430    False
5970    False
7720    False
4330    False
6710    False
2940    False
4650    False
5370    False
5480    False
6750    False
6760    False
2510    False
2590    False
2910    False
2970    False
3260    False
3370    False
3380    False
3520    False
3630    False
3690    False
3760    False
3900    False
3980    False
4250    False
4320    False
4720    False
5390    False
6310    False
6440    False
6930    False
7490    False
7880    False
8021    False
2580    False
2830    False
3040    False
3110    False
3530    False
3720    False
4350    False
4570    False
6880    False
6880    False
7050    Fals

So let's do some exploration.

In [29]:
print(crimes_extract['UTMPoint'])

0    POINT (436025.3086861699 4644953.17679981)
1    POINT (436377.940881997 4643860.108357577)
2    POINT (436836.8507415869 4645093.21940705)
3    POINT (436372.3575166143 4643442.41025281)
4    POINT (438086.3348712226 4643297.20981963)
Name: UTMPoint, dtype: object


I want to grep the point coordinates from the schools dataset, but can't do that directly from the geopandas display, so this is to muck about with stdout.

In [39]:
from io import StringIO
import sys

try:
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    print(schools['UTMPoint'])
except Exception as e:
    print("An exception happened: {0}".format(e))
finally:
    sys.stdout = old_stdout

# examine mystdout.getvalue()
output_text = mystdout.getvalue()
output_text
"436025.3086861699" in output_text

False

Well, that would appear to be a problem. The point I'm looking for isn't in the school dataset.

When looking above to grab the returned point, I noticed that while the crimes starting point is in UTM, the returned school point is in geodesic. Let's try renaming columns in the schools dataset to see if that helps.

In [46]:
schools_renamed = schools.rename(columns={'geometry': 'geodesic geometry', 'UTMPoint': 'geometry'})
schools_renamed.head()

,geodesic geometry,SCHOOL_NM,SCHOOL_ID,SCH_ADDR,GRADE_CAT,SCH_TYPE,lon,lat,UNIT_ID,UTMx,UTMy,geometry
UNIT_ID,,,,,,,,,,,,
4610,POINT (-87.5748539059 41.7483660139),MANN,610052,8050 S CHAPPEL AV,ES,Elementary School,-87.574854,41.748366,4610,452204.771983,4.621998e+06,POINT (452204.7719826915 4621997.860158128)
5180,POINT (-87.57284397069999 41.768650177),OKEEFFE,610103,6940 S MERRILL AV,ES,Elementary School,-87.572844,41.768650,5180,452386.881166,4.624249e+06,POINT (452386.881166386 4624248.790945017)
5300,POINT (-87.5829920307 41.768594283),PARKSIDE,610116,6938 S EAST END AV,ES,Elementary School,-87.582992,41.768594,5300,451543.358669,4.624248e+06,POINT (451543.3586693273 4624248.25277694)
5440,POINT (-87.60800008530001 41.740365734),PIRIE,610130,650 E 85TH ST,ES,Elementary School,-87.608000,41.740366,5440,449442.603124,4.621129e+06,POINT (449442.6031235668 4621128.574748854)
5610,POINT (-87.60015764629999 41.764223475),REVERE,610146,1010 E 72ND ST,ES,Elementary School,-87.600158,41.764223,5610,450113.209645,4.623773e+06,POINT (450113.2096448886 4623772.796898597)


In [47]:
schools_unary_union = schools_renamed.unary_union

In [75]:
crimes_extract['nearest_id'] = crimes_extract.apply(distance_to_nearest,
                                                    geom_union=schools_unary_union,
                                                    df1=crimes_extract,
                                                    df2=schools_renamed,
                                                    geom1_col='UTMPoint',
                                                    geom2_col='geometry',
                                                    src_column='UNIT_ID',
                                                    axis=1
                                                   )
crimes_extract

/Users/jon/anaconda3/envs/geospatial/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ID,case number,date,block,iucr,primary type,desc,locdesc,arrest,domestic,...,year,updated on,lat,lon,location,community,UTMx,UTMy,UTMPoint,nearest_id
0,8265244,HT498458,09/15/2011 08:00:00 AM,040XX N MENARD AVE,0620,BURGLARY,UNLAWFUL ENTRY,RESIDENCE,True,False,...,2011,02/12/2018 03:46:59 PM,41.953965,-87.771918,"(41.95396528, -87.771918163)",PORTAGE PARK,436025.308686,4.644953e+06,POINT (436025.3086861699 4644953.17679981),"(6190, 762.6890819951975)"
1,7357772,HS159408,07/01/2001 10:00:00 AM,056XX W CORNELIA AVE,0840,THEFT,FINANCIAL ID THEFT: OVER $300,RESIDENCE,False,False,...,2001,02/18/2010 01:12:55 AM,41.944150,-87.767546,"(41.9441497, -87.767545572)",PORTAGE PARK,436377.940882,4.643860e+06,POINT (436377.940881997 4643860.108357577),"(5600, 175.30589971921285)"
2,7609828,HS413731,07/13/2010 12:00:00 PM,041XX N LONG AVE,0820,THEFT,$500 AND UNDER,PARK PROPERTY,False,False,...,2010,02/04/2016 06:33:39 AM,41.955292,-87.762142,"(41.95529191, -87.762141971)",PORTAGE PARK,436836.850742,4.645093e+06,POINT (436836.8507415869 4645093.21940705),"(5490, 213.4040849657432)"
3,7610439,HS413034,07/16/2010 03:00:00 AM,056XX W SCHOOL ST,0486,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,...,2010,02/04/2016 06:33:39 AM,41.940387,-87.767568,"(41.940387482, -87.767567805)",PORTAGE PARK,436372.357517,4.643442e+06,POINT (436372.3575166143 4643442.41025281),"(5600, 308.14711617099)"
4,7610466,HS414753,07/17/2010 12:25:00 AM,032XX N CICERO AVE,1506,PROSTITUTION,SOLICIT ON PUBLIC WAY,ALLEY,True,False,...,2010,02/04/2016 06:33:39 AM,41.939216,-87.746878,"(41.939216164, -87.746877814)",PORTAGE PARK,438086.334871,4.643297e+06,POINT (438086.3348712226 4643297.20981963),"(3270, 436.0276658105222)"


Well crud. That didn't work the way I'd expected. Let's pull it apart again.

In [50]:
print(nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[0])
print(nearest_points(crimes_extract.iloc[0]['UTMPoint'], schools_unary_union)[1])

POINT (436025.3086861699 4644953.17679981)
POINT (435543.9217422368 4644361.601575511)


So that looks better. We've got UTM-based points now at least. Let's try looking inside the schools dataset for that point.

In [68]:
try:
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    print(schools_renamed)
except Exception as e:
    print("An exception happened: {0}".format(e))
finally:
    sys.stdout = old_stdout

# examine mystdout.getvalue()
matched_coord = "435543.9217422368 4644361.601575511"
output_text = mystdout.getvalue()
#print(output_text)
print("435543.9217422368 4644361.601575511" in output_text)
output_lines = output_text.splitlines()
[line for line in output_lines if matched_coord in line]

True


['6190     POINT (435543.9217422368 4644361.601575511)  ']

Hmm, now I'm confused. That coordinate pair is in the schools_renamed dataset. Why aren't we finding it?

In [61]:
renamed_real_nearest = schools['geometry'] == nearest_points(crimes_extract.iloc[0]['UTMPoint'],
                                                             schools_unary_union)[1]
renamed_real_nearest

UNIT_ID
4610    False
5180    False
5300    False
5440    False
5610    False
5670    False
5880    False
6100    False
6350    False
6900    False
1135    False
2123    False
2123    False
1105    False
6320    False
6450    False
6570    False
7190    False
7610    False
5640    False
6490    False
4230    False
4310    False
4430    False
4430    False
4710    False
5430    False
5970    False
7720    False
4330    False
6710    False
2940    False
4650    False
5370    False
5480    False
6750    False
6760    False
2510    False
2590    False
2910    False
2970    False
3260    False
3370    False
3380    False
3520    False
3630    False
3690    False
3760    False
3900    False
3980    False
4250    False
4320    False
4720    False
5390    False
6310    False
6440    False
6930    False
7490    False
7880    False
8021    False
2580    False
2830    False
3040    False
3110    False
3530    False
3720    False
4350    False
4570    False
6880    False
6880    False
7050    Fals

Too many lines to manually search. Time to capture the output again.

In [69]:
try:
    old_stdout = sys.stdout
    sys.stdout = mystdout = StringIO()
    print(renamed_real_nearest)
except Exception as e:
    print("An exception happened: {0}".format(e))
finally:
    sys.stdout = old_stdout

# examine mystdout.getvalue()
output_text = mystdout.getvalue()
output_lines = output_text.splitlines()
print(matched_coord in output_text)
result = [line for line in output_lines if matched_coord in line]
print(result)

False
[]


What's going on? Let's manually pull up the school matched above, UNIT_ID 6190

In [72]:
print(schools_renamed[schools_renamed['UNIT_ID'] == 6190]['geometry'])
print(matched_coord)

UNIT_ID
6190    POINT (435543.9217422368 4644361.601575511)
Name: geometry, dtype: object
435543.9217422368 4644361.601575511


Well, that certainly looks like the same point to me. Let's try constructing a new Point and compare the two.

In [76]:
matched_point = shapely.geometry.Point(435543.9217422368, 4644361.601575511)
schools_renamed[schools_renamed['UNIT_ID'] == 6190]['geometry'] == matched_point

UNIT_ID
6190    True
Name: geometry, dtype: bool

In [77]:
schools_renamed.to_pickle('schools_renamed.pkl')

In [78]:
schools[schools['UNIT_ID'] == 6190]['UTMPoint'] == matched_point

UNIT_ID
6190    True
Name: UTMPoint, dtype: bool